# Geo Market Insights — End-to-End (Colab)

This notebook runs the full pipeline:
1. Install requirements (Colab)
2. Generate an H3 grid over Wilmington, DE area
3. Build synthetic demand (replace with real ACS if desired)
4. Compute competition pressure from `data/inputs/competitors.csv`
5. Score and rank candidate sites
6. Render an interactive Folium map


In [ ]:
import sys, os
if 'google.colab' in sys.modules:
    !pip -q install -r /content/geo-market-insights/requirements.txt
else:
    print('Running outside Colab; ensure requirements are installed.')

In [ ]:
# If on Colab, clone or mount your repo so this path exists:
import sys, os
BASE = '/content/geo-market-insights'
if not os.path.exists(BASE):
    # Assume this notebook was opened from GitHub; clone the same repo path (edit to your username)
    !git clone https://github.com/your-username/geo-market-insights.git {BASE}
os.chdir(BASE)
print('Working dir:', os.getcwd())

In [ ]:
import pandas as pd
import geopandas as gpd
from src.config import CENTER_LAT, CENTER_LON, RADIUS_M, H3_RESOLUTION, WEIGHT_DEMAND, WEIGHT_COMPETITION, COMPETITION_DECAY_M, SEED
from src.geo_utils import hexes_in_radius
from src.demand import synthetic_demand_for_hexes
from src.competition import load_competitors, competition_pressure_for_hexes
from src.scoring import score_sites
from src.visualize import render_map

# 1) Grid
hex_gdf = hexes_in_radius(CENTER_LAT, CENTER_LON, RADIUS_M, H3_RESOLUTION)
print('Hex count:', len(hex_gdf))

# 2) Demand
hex_dem = synthetic_demand_for_hexes(hex_gdf, seed=SEED)

# 3) Competition
competitors = load_competitors('data/inputs/competitors.csv')
hex_comp = competition_pressure_for_hexes(hex_dem, competitors, decay_m=COMPETITION_DECAY_M)

# 4) Score
hex_scored = score_sites(hex_comp, WEIGHT_DEMAND, WEIGHT_COMPETITION)
hex_scored.head()

In [ ]:
import os
os.makedirs('data/outputs', exist_ok=True)
hex_scored[['h3','demand','competition','site_score']].to_csv('data/outputs/candidates_ranked.csv', index=False)
out_map = render_map(hex_scored, competitors, out_path_html='data/outputs/market_map.html', top_n=15)
out_map

### Next Steps
- Replace synthetic demand with **real ACS data** (median income, pop density, households) and re-score.
- Add **existing store list** separately and penalize self-cannibalization more than competitors.
- Switch to **travel-time** distances (OSRM/Valhalla) for more realism.
